In [42]:
df_silver = spark.read.table("lh_silver.slv_redata_demanda_evolucion_day_cleaning")

display(df_silver.limit(5))
print(f"Total de filas iniciales: {df_silver.count():,}")

StatementMeta(, f080ca1d-4630-4c9e-9c21-4712dba7faad, 44, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 51eb99a0-1c3c-4ad2-b83c-f2e4a31f7f5e)

Total de filas iniciales: 5,545


In [43]:
from pyspark.sql import functions as F

# Contar cuántos valores no nulos tiene cada columna
non_null_counts = df_silver.select([
    F.count(F.col(c)).alias(c)
    for c in df_silver.columns
]).collect()[0].asDict()

# Filtrar las columnas que tienen al menos un valor no nulo
cols_to_keep = [c for c, count in non_null_counts.items() if count > 0]

# Crear nuevo DataFrame sin las columnas completamente nulas
df_no_null_cols = df_silver.select(cols_to_keep)

print(f"Columnas eliminadas: {[c for c in df_silver.columns if c not in cols_to_keep]}")
print(f"Total de columnas finales: {len(df_no_null_cols.columns)}")

# Mostrar el resultado
display(df_no_null_cols.limit(5))


StatementMeta(, f080ca1d-4630-4c9e-9c21-4712dba7faad, 45, Finished, Available, Finished)

Columnas eliminadas: ['ccaa_name', 'series_group_id', 'series_description', 'series_icon', 'series_attribute_type', 'series_magnitude']
Total de columnas finales: 14


SynapseWidget(Synapse.DataFrame, 7504d798-6cb6-4f96-a607-bae83b958078)

In [44]:
# Eliminar duplicados según todas las columnas excepto ingestion_timestamp
df_clean = df_no_null_cols.dropDuplicates([
    "geo_id", "geo_name", "geo_limit", "time_trunc",
    "series_type", "series_id", "series_title",
    "series_color", "is_composite", "series_last_update", "datetime", "value", "percentage"
])

# Mostrar resultado y contar
display(df_clean.limit(5))
print(f"Total después de limpiar duplicados: {df_clean.count():,}")


StatementMeta(, f080ca1d-4630-4c9e-9c21-4712dba7faad, 46, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e43159bb-97db-4ed0-a4e7-fb9bbe2266a8)

Total después de limpiar duplicados: 3,705


In [45]:
cols = ["geo_id", "geo_name", "geo_limit", "time_trunc", "series_type", "series_id", 
        "series_title", "series_color", "is_composite", "percentage"]

for c in cols:
    uniques = [row[c] for row in df_clean.select(c).distinct().collect()]
    count_uniques = df_clean.select(c).distinct().count()
    print(f"\n📌 Valores únicos en '{c}': {count_uniques}")
    print(uniques)


StatementMeta(, f080ca1d-4630-4c9e-9c21-4712dba7faad, 47, Finished, Available, Finished)


📌 Valores únicos en 'geo_id': 5
[8743, 8745, 8741, 8742, 8744]

📌 Valores únicos en 'geo_name': 5
['Comunidad de Ceuta', 'Islas Canarias', 'Islas Baleares', 'Comunidad de Melilla', 'Península']

📌 Valores únicos en 'geo_limit': 5
['ceuta', 'baleares', 'canarias', 'peninsular', 'melilla']

📌 Valores únicos en 'time_trunc': 1
['day']

📌 Valores únicos en 'series_type': 1
['Demanda']

📌 Valores únicos en 'series_id': 1
['10297']

📌 Valores únicos en 'series_title': 1
['Demanda']

📌 Valores únicos en 'series_color': 1
['#ffea00']

📌 Valores únicos en 'is_composite': 1
[False]

📌 Valores únicos en 'percentage': 1
[1.0]


In [46]:
df_clean = df_clean.drop("time_trunc", 'series_type', 'series_id', 'series_title', "series_color", "is_composite", "percentage", "ingestion_timestamp")

display(df_clean.limit(2))

StatementMeta(, f080ca1d-4630-4c9e-9c21-4712dba7faad, 48, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1663b4ff-c0c6-4c70-b370-57ec6591ac69)

In [47]:
# Contamos antes
before_count = df_clean.count()

# Eliminamos filas con nulos en columnas esenciales
df_clean = df_clean.na.drop(subset=["datetime", "value", "geo_name"])

# Contamos después
after_count = df_clean.count()

# Mostramos resultados según lo ocurrido
if after_count < before_count:
    print(f"Se eliminaron {before_count - after_count:,} filas con valores nulos. Total filas tras la limpieza {after_count}.")
else:
    print("No se eliminaron filas; no había valores nulos en las columnas esenciales.")


StatementMeta(, f080ca1d-4630-4c9e-9c21-4712dba7faad, 49, Finished, Available, Finished)

No se eliminaron filas; no había valores nulos en las columnas esenciales.


In [48]:
# convertir columnas al tipo correcto
df_clean = (df_clean
    .withColumn("datetime", F.to_timestamp("datetime"))
    .withColumn("value", F.col("value").cast("double"))
)

print("Finalizada la conversión de las columnas 'datetime' y 'value'.")

StatementMeta(, f080ca1d-4630-4c9e-9c21-4712dba7faad, 50, Finished, Available, Finished)

Finalizada la conversión de las columnas 'datetime' y 'value'.


In [50]:
# Crear la columna "year" extrayendo el año de "datetime"
df_clean = df_clean.withColumn("year", F.year(F.col("datetime")))

display(df_clean.limit(3))

StatementMeta(, f080ca1d-4630-4c9e-9c21-4712dba7faad, 52, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3b3ae9dd-17c7-4d1e-a98c-5b417a3beaec)

In [52]:
# guardamos la tabla limpia en la capa Silver
df_clean.write.mode("overwrite").saveAsTable("lh_silver.slv_redata_demanda_evolucion_day_cleaned")

print("✅ Tabla guardada correctamente en lh_silver.slv_redata_demanda_evolucion_day_cleaned")

StatementMeta(, f080ca1d-4630-4c9e-9c21-4712dba7faad, 54, Finished, Available, Finished)

✅ Tabla guardada correctamente en lh_silver.slv_redata_demanda_evolucion_day_cleaned
